In [ ]:
import os
from pathlib import Path
from decimal import ROUND_HALF_UP, Decimal

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter('ignore')

In [ ]:
def calc_target_shift2(price):
    price.loc[:, 'Target_shift2'] = price['Close'].pct_change()
    return price

def add_columns_per_code(price):
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(calc_target_shift2)
    price = price.reset_index(drop=True)
    return price

def preprocessor(price):
    price = add_columns_per_code(price)
    return price

In [ ]:
# Public LB Score → 0.121
def predictor(feature_df):
    return feature_df['Target_shift2']

# Public LB Score → 5.434
# def predictor(feature_df):
#     return feature_df['Target_shift2'].where(feature_df['Target'].isnull(), feature_df['Target'])

In [ ]:
def add_rank(df, col_name="pred"):
    df["Rank"] = df.groupby("Date")[col_name].rank(ascending=False, method="first") - 1 
    df["Rank"] = df["Rank"].astype("int")
    return df

In [ ]:
past_df = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv')
past_df = pd.concat([past_df, pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv')])

In [ ]:
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

In [ ]:
for i, (prices, options, financials, trades, secondary_prices, sample_prediction) in enumerate(iter_test):
    current_date = prices["Date"].iloc[0]

    threshold = (pd.Timestamp(current_date) - pd.offsets.BDay(30)).strftime("%Y-%m-%d")
    past_df = past_df[past_df["Date"] >= threshold]

    if (past_df['Date'] == current_date).sum() == 0:
        past_df = pd.concat([past_df, prices]).reset_index(drop=True)
        
    feature_df = preprocessor(past_df)
    feature_df = feature_df[feature_df['Date'] == current_date]

    feature_df["pred"] = predictor(feature_df)

    feature_df = add_rank(feature_df)
    feature_map = feature_df.set_index('SecuritiesCode')['Rank'].to_dict()
    sample_prediction['Rank'] = sample_prediction['SecuritiesCode'].map(feature_map)

    env.predict(sample_prediction)